In [1]:
import warnings
warnings.filterwarnings('ignore')

from configparser import ConfigParser

from typing import Callable

import gym
import gym_anytrading

from gym_anytrading.envs import CryptoEnvLogINFER

from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3 import A2C, PPO, DQN
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.utils import set_random_seed

#from stable_baselines3.dqn.policies import MlpPolicy
from stable_baselines3.ppo.policies import MlpPolicy

import tensorflow as tf

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
import datetime

import multiprocessing

import quantstats as qs

import sqlite3
from sqlalchemy import create_engine

In [2]:
configur = ConfigParser()
config_file_name = "config_14a_mix_infer_s.ini"
print (configur.read(os.path.join('../../simulations/config_files', config_file_name)))

['../../simulations/config_files/config_14a_mix_infer_s.ini']


In [3]:
config_file_name_without_extension = config_file_name.replace(".ini", "")

ccy = configur.get('data', 'ccy')

data_frequency_train = configur.get('data', 'data_frequency_train')
data_frequency_val = configur.get('data', 'data_frequency_val')

start_date_train = configur.get('environment', 'start_date')

window_size = configur.getint('environment', 'window_size')
list_features = configur.get('environment', 'features').split(',')
target = configur.get('environment', 'target')

In [4]:
# model_name = "DQN"
# # the saved model does not contain the replay buffer
# loaded_model = DQN.load("../../simulations/save_models_components/model/model_DQN_"+ config_file_name_without_extension + ".zip")
# print(f"The loaded_model has {loaded_model.replay_buffer.size()} transitions in its buffer")

# # load it into the loaded_model
# loaded_model.load_replay_buffer("../../simulations/save_models_components/replay_buffer/replay_buffer_DQN_" + config_file_name_without_extension + ".pkl")

# # now the loaded replay is not empty anymore
# print(f"The loaded_model has {loaded_model.replay_buffer.size()} transitions in its buffer")

# saved_policy = MlpPolicy.load("../../simulations/save_models_components/policy/policy_DQN_" + config_file_name_without_extension)

In [5]:
model_name = "PPO"
# the saved model does not contain the replay buffer
loaded_model = PPO.load("../../simulations/save_models_components/model/model_PPO_"+ config_file_name_without_extension + ".zip")

saved_policy = MlpPolicy.load("../../simulations/save_models_components/policy/policy_PPO_" + config_file_name_without_extension)

In [6]:
mid_date='2023-01-01 00:00:00'
end_date='2023-02-14 00:00:00'

In [7]:
def get_and_process_data(ccy, data_frequency, start_date, end_date):
    try:
        
        # Get the data
        db_address = 'sqlite:///../../data/db/crypto.db'
        engine = create_engine(db_address, echo=False)
        sqlite_connection = engine.connect()

        input_tbl_name = "tbl_all_features" + "_" + ccy + "_" + data_frequency
        
        sql_command = "SELECT * FROM " + input_tbl_name

        if(start_date!="" and end_date!=""):
            
            sd = datetime.datetime.strptime(start_date,'%d %b, %Y').strftime('%Y-%m-%d')
            ed = datetime.datetime.strptime(end_date,'%d %b, %Y').strftime('%Y-%m-%d')

            sql_append = " WHERE " + "date(date) >= " + "'" + sd + "'" + " AND date(date) <= " + "'" + ed + "'"
            sql_command = sql_command + sql_append

        df = pd.read_sql(sql_command, sqlite_connection)

        sqlite_connection.close()

        # Converting Date Column to DateTime Type
        # Set the index on the dataframe
        df['date_index'] = df['date']
        df.set_index('date_index', inplace=True)

        return df

    except Exception as e:
        print(e)

In [8]:
def standardize_data(df, list_columns):
    for col in list_columns:
        df[col] = (df[col] - df[col].mean()) / df[col].std()
    return df

In [9]:
# Format the data
start_date_temp = datetime.datetime.strptime(mid_date,'%Y-%m-%d %H:%M:%S')
start_date_temp2 = start_date_temp + datetime.timedelta(days=-(window_size+1))
start_date_db = start_date_temp2.strftime('%d %b, %Y')
end_date_db = datetime.datetime.strptime(end_date,'%Y-%m-%d %H:%M:%S').strftime('%d %b, %Y')

In [10]:
df_val = get_and_process_data(ccy, data_frequency_val, start_date_db, end_date_db)
df_val = standardize_data(df_val, list_features)

In [11]:
mid_date_id_val = int(df_val.index.get_loc(datetime.datetime.strptime(mid_date,'%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d %H:%M:%S.%f')))
end_date_id_val = int(df_val.index.get_loc(datetime.datetime.strptime(end_date,'%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d %H:%M:%S.%f')))

In [12]:
def my_process_data(env):
    start = env.frame_bound[0] - env.window_size
    end = env.frame_bound[1]
    prices = env.df.loc[:, target].to_numpy()[start:end]
    signal_features = env.df.loc[:, list_features].to_numpy()[start:end]
    dates = env.df.index.to_numpy()[start:end]
    return prices, signal_features, dates

class MyEnv(CryptoEnvLogINFER):
    _process_data = my_process_data

In [13]:
env = MyEnv(df=df_val, frame_bound=(mid_date_id_val,end_date_id_val), window_size=window_size)

In [14]:
num_of_simulations=1000

In [15]:
df_sim_results = pd.DataFrame()
df_robot_actions_and_env_final = pd.DataFrame()

df_sim_results_temp = pd.DataFrame(columns=['total_reward_cash', 'total_profit_percentage', 'fmt_total_profit_percentage', 'num_of_trades'])
df_robot_actions_and_env_temp = pd.DataFrame(columns=['dates', 'prices', 'actions', 'total_reward_cash', 'total_profit_percentage', 'fmt_total_profit_percentage', 'num_of_trades', 'sim_id'])

list_sim_id = []

sim_id = 1
for i in range(num_of_simulations):   

    #Setting up the Agent Environment
    obs = env.reset()

    list_sim_id.append(sim_id)

    while True: 
        obs = obs[np.newaxis, ...]

        action, _states = loaded_model.predict(obs)
        
        obs, rewards, done, info = env.step(action)
        
        if done:
            df_sim_results_temp = df_sim_results_temp.append(info, ignore_index=True, sort=False)
            break

    # Export robot actions plot and data
    fig, df_robot_actions = env.render_all()
    # fig.write_html(os.path.join('results', 'plots', 'robot_actions', 'robot_actions_' + config_file_name_without_extension + '_sim_id_' + str(sim_id) + '.html'))

    # Export the history of details (the info dict inside of the environment)
    df_sim_env_data = pd.DataFrame.from_dict(env.history)

    # Merge robot actions data + sim env data
    df_sim_env_data_initial = pd.DataFrame(columns=['total_reward_cash', 'total_profit_percentage', 'fmt_total_profit_percentage', 'num_of_trades'])
    for i in range(0, window_size + 1):
        df_sim_env_data_initial = df_sim_env_data_initial.append(pd.Series([0, 0, 0, 0], index=df_sim_env_data_initial.columns), ignore_index=True)
    
    df_sim_env_data_temp = df_sim_env_data_initial.append(df_sim_env_data, ignore_index=True)
    
    df_robot_actions_and_env_temp = pd.merge(df_robot_actions, df_sim_env_data_temp, left_index=True, right_index=True, how = "outer")
    df_robot_actions_and_env_temp['sim_id'] = sim_id

    df_robot_actions_and_env_final = pd.concat([df_robot_actions_and_env_final, df_robot_actions_and_env_temp], axis=0)

    sim_id = sim_id + 1

df_sim_results_temp['sim_id'] = list_sim_id
df_sim_results_temp['ccy'] = ccy
df_sim_results_temp['data_frequency_train'] = data_frequency_train
df_sim_results_temp['data_frequency_val'] = data_frequency_val
df_sim_results_temp['window_size'] = window_size
df_sim_results_temp['start_date'] = start_date_train
df_sim_results_temp['mid_date'] = mid_date
df_sim_results_temp['end_date'] = end_date
df_sim_results_temp['target'] = target
df_sim_results_temp['list_features'] = str(list_features)
df_sim_results_temp['model_type'] = model_name 
df_sim_results_temp['num_of_simulations'] = num_of_simulations   

df_sim_results = pd.concat([df_sim_results, df_sim_results_temp], axis=0)

In [16]:
df_sim_results = df_sim_results.reset_index(drop=True)
df_sim_results = df_sim_results.drop_duplicates()

## SIMULATION RESULTS

# Format data type
df_sim_results['start_date'] = pd.to_datetime(df_sim_results['start_date'])
df_sim_results['mid_date'] = pd.to_datetime(df_sim_results['mid_date'])
df_sim_results['end_date'] = pd.to_datetime(df_sim_results['end_date'])

# Write to the db
db_address = 'sqlite:///../results/data/db/trader.db'
engine = create_engine(db_address, echo=False)
sqlite_connection = engine.connect()

output_tbl_name = "tbl_trader_bernardo_simulation_results_" + config_file_name_without_extension

df_sim_results.to_sql(output_tbl_name, sqlite_connection, if_exists='replace', index=False)

sqlite_connection.close()

## ROBOT ACTIONS and ENV RESULTS

# Format data type
df_robot_actions_and_env_final['dates'] = pd.to_datetime(df_robot_actions_and_env_final['dates'])
df_robot_actions_and_env_final['total_reward_cash'] = df_robot_actions_and_env_final['total_reward_cash'].astype(float)
df_robot_actions_and_env_final['total_profit_percentage'] = df_robot_actions_and_env_final['total_profit_percentage'].astype(float)
df_robot_actions_and_env_final['fmt_total_profit_percentage'] = df_robot_actions_and_env_final['fmt_total_profit_percentage'].astype(float)
df_robot_actions_and_env_final['num_of_trades'] = df_robot_actions_and_env_final['num_of_trades'].astype(float)

# Write to the db
db_address = 'sqlite:///../results/data/db/trader.db'
engine = create_engine(db_address, echo=False)
sqlite_connection = engine.connect()

output_tbl_name = "tbl_trader_bernardo_robot_actions_and_env_" + config_file_name_without_extension

df_robot_actions_and_env_final.to_sql(output_tbl_name, sqlite_connection, if_exists='replace', index=False)

sqlite_connection.close()

## Get Data

In [17]:
def get_and_process_trader_data(config_file_name_without_extension):
    try:
        
        # Get the data
        db_address = 'sqlite:///../results/data/db/trader.db'
        engine = create_engine(db_address, echo=False)
        sqlite_connection = engine.connect()

        input_tbl_name = "tbl_trader_bernardo_simulation_results" + "_" + config_file_name_without_extension
        
        sql_command = "SELECT * FROM " + input_tbl_name

        df = pd.read_sql(sql_command, sqlite_connection)

        sqlite_connection.close()

        return df

    except Exception as e:
        print(e)

In [18]:
df_trader_results = get_and_process_trader_data(config_file_name_without_extension)

In [19]:
fig = px.box(df_trader_results, y="fmt_total_profit_percentage", color="model_type", points="all", color_discrete_sequence=[ "#FF7F0E", "#00CC96", "#10aded", "#8A56EF"],  width=800, height=600)
fig.show()

In [20]:
fig = px.histogram(df_trader_results, x="fmt_total_profit_percentage", color="model_type", color_discrete_sequence=[ "#FF7F0E", "#00CC96", "#10aded", "#8A56EF"],  width=800, height=600, marginal="rug", # can be `box`, `violin`
                         hover_data=['total_reward_cash','fmt_total_profit_percentage', 'num_of_trades', 'sim_id'])
fig.show()

In [21]:
df_trader_results_stats = df_trader_results.groupby(['model_type'])['fmt_total_profit_percentage'].agg(['median']).reset_index()

In [22]:
# i_A2C = np.argmin(np.abs(df_trader_results[df_trader_results['model_type']=='A2C']['fmt_total_profit_percentage'] - df_trader_results_stats[df_trader_results_stats['model_type']=='A2C']['median'].values[0]))
# median_sim_id_A2C = df_trader_results[df_trader_results['model_type']=='A2C'].iloc[i_A2C]['sim_id']

i_PPO = np.argmin(np.abs(df_trader_results[df_trader_results['model_type']=='PPO']['fmt_total_profit_percentage'] - df_trader_results_stats[df_trader_results_stats['model_type']=='PPO']['median'].values[0]))
median_sim_id_PPO = df_trader_results[df_trader_results['model_type']=='PPO'].iloc[i_PPO]['sim_id']

# i_DQN = np.argmin(np.abs(df_trader_results[df_trader_results['model_type']=='DQN']['fmt_total_profit_percentage'] - df_trader_results_stats[df_trader_results_stats['model_type']=='DQN']['median'].values[0]))
# median_sim_id_DQN = df_trader_results[df_trader_results['model_type']=='DQN'].iloc[i_DQN]['sim_id']

# i_RANDOM = np.argmin(np.abs(df_trader_results[df_trader_results['model_type']=='RANDOM']['fmt_total_profit_percentage'] - df_trader_results_stats[df_trader_results_stats['model_type']=='RANDOM']['median'].values[0]))
# median_sim_id_RANDOM = df_trader_results[df_trader_results['model_type']=='RANDOM'].iloc[i_RANDOM]['sim_id']

In [23]:
def get_and_process_trader_robot_actions_and_env_data_one_sim_id(config_file_name_without_extension, sim_id):
    try:
        
        # Get the data
        db_address = 'sqlite:///../results/data/db/trader.db'
        engine = create_engine(db_address, echo=False)
        sqlite_connection = engine.connect()

        input_tbl_name = "tbl_trader_bernardo_robot_actions_and_env" + "_" + config_file_name_without_extension
        
        sql_command = "SELECT * FROM " + input_tbl_name
        
        sql_append = " WHERE " + "sim_id = " + str(sim_id)
        
        sql_command = sql_command + sql_append

        df = pd.read_sql(sql_command, sqlite_connection)

        sqlite_connection.close()

        return df

    except Exception as e:
        print(e)

In [24]:
def plot_robot_actions_all_models(config_file_name_without_extension, sim_id_A2C, sim_id_PPO, sim_id_DQN, sim_id_RANDOM):
    
    # df_robot_actions_sim_id_A2C = get_and_process_trader_robot_actions_and_env_data_one_sim_id(config_file_name_without_extension, sim_id_A2C)
    df_robot_actions_sim_id_PPO = get_and_process_trader_robot_actions_and_env_data_one_sim_id(config_file_name_without_extension, sim_id_PPO)
    # df_robot_actions_sim_id_DQN = get_and_process_trader_robot_actions_and_env_data_one_sim_id(config_file_name_without_extension, sim_id_DQN)
    # df_robot_actions_sim_id_RANDOM = get_and_process_trader_robot_actions_and_env_data_one_sim_id(config_file_name_without_extension, sim_id_RANDOM)

    # df_simulation_sim_id_A2C = df_trader_results[df_trader_results['sim_id'] == sim_id_A2C]
    df_simulation_sim_id_PPO = df_trader_results[df_trader_results['sim_id'] == sim_id_PPO]
    # df_simulation_sim_id_DQN = df_trader_results[df_trader_results['sim_id'] == sim_id_DQN]
    # df_simulation_sim_id_RANDOM = df_trader_results[df_trader_results['sim_id'] == sim_id_RANDOM]

    # str_title_A2C = f"sim_id: {sim_id_A2C} | model_type: {df_simulation_sim_id_A2C.model_type.values[0]} | Total Profit percentage {round(df_simulation_sim_id_A2C.fmt_total_profit_percentage.values[0],2)}%"
    str_title_PPO = f"sim_id: {sim_id_PPO} | model_type: {df_simulation_sim_id_PPO.model_type.values[0]} | Total Profit percentage {round(df_simulation_sim_id_PPO.fmt_total_profit_percentage.values[0],2)}%"
    # str_title_DQN = f"sim_id: {sim_id_DQN} | model_type: {df_simulation_sim_id_DQN.model_type.values[0]} | Total Profit percentage {round(df_simulation_sim_id_DQN.fmt_total_profit_percentage.values[0],2)}%"
    # str_title_RANDOM = f"sim_id: {sim_id_RANDOM} | model_type: {df_simulation_sim_id_RANDOM.model_type.values[0]} | Total Profit percentage {round(df_simulation_sim_id_RANDOM.fmt_total_profit_percentage.values[0],2)}%"

    str_title = "The median profit percentage results per model"

    fig = make_subplots(rows=4, cols=1, subplot_titles=("", str_title_PPO, "", ""), shared_xaxes='all', shared_yaxes='all')

    # fig.add_trace(go.Scatter(name='Price', x=df_robot_actions_sim_id_A2C["dates"], y=df_robot_actions_sim_id_A2C["prices"], mode='lines', marker_color='#969696', legendgroup = '1'), row=1, col=1)
    # fig.add_trace(go.Scatter(name='Long', x=df_robot_actions_sim_id_A2C[df_robot_actions_sim_id_A2C['actions'] == 1]['dates'], y=df_robot_actions_sim_id_A2C[df_robot_actions_sim_id_A2C['actions'] == 1]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#228b22', legendgroup = '1'), row=1, col=1)
    # fig.add_trace(go.Scatter(name='Short', x=df_robot_actions_sim_id_A2C[df_robot_actions_sim_id_A2C['actions'] == 0]['dates'], y=df_robot_actions_sim_id_A2C[df_robot_actions_sim_id_A2C['actions'] == 0]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#c83232', legendgroup = '1'), row=1, col=1)

    fig.add_trace(go.Scatter(name='Price', x=df_robot_actions_sim_id_PPO["dates"], y=df_robot_actions_sim_id_PPO["prices"], mode='lines', marker_color='#969696', legendgroup = '2'), row=2, col=1)
    fig.add_trace(go.Scatter(name='Long', x=df_robot_actions_sim_id_PPO[df_robot_actions_sim_id_PPO['actions'] == 1]['dates'], y=df_robot_actions_sim_id_PPO[df_robot_actions_sim_id_PPO['actions'] == 1]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#228b22', legendgroup = '2'), row=2, col=1)
    fig.add_trace(go.Scatter(name='Short', x=df_robot_actions_sim_id_PPO[df_robot_actions_sim_id_PPO['actions'] == 0]['dates'], y=df_robot_actions_sim_id_PPO[df_robot_actions_sim_id_PPO['actions'] == 0]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#c83232', legendgroup = '2'), row=2, col=1)

    # fig.add_trace(go.Scatter(name='Price', x=df_robot_actions_sim_id_DQN["dates"], y=df_robot_actions_sim_id_DQN["prices"], mode='lines', marker_color='#969696', legendgroup = '3'), row=3, col=1)
    # fig.add_trace(go.Scatter(name='Long', x=df_robot_actions_sim_id_DQN[df_robot_actions_sim_id_DQN['actions'] == 1]['dates'], y=df_robot_actions_sim_id_DQN[df_robot_actions_sim_id_DQN['actions'] == 1]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#228b22', legendgroup = '3'), row=3, col=1)
    # fig.add_trace(go.Scatter(name='Short', x=df_robot_actions_sim_id_DQN[df_robot_actions_sim_id_DQN['actions'] == 0]['dates'], y=df_robot_actions_sim_id_DQN[df_robot_actions_sim_id_DQN['actions'] == 0]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#c83232', legendgroup = '3'), row=3, col=1)

    # fig.add_trace(go.Scatter(name='Price', x=df_robot_actions_sim_id_RANDOM["dates"], y=df_robot_actions_sim_id_RANDOM["prices"], mode='lines', marker_color='#969696', legendgroup = '4'), row=4, col=1)
    # fig.add_trace(go.Scatter(name='Long', x=df_robot_actions_sim_id_RANDOM[df_robot_actions_sim_id_RANDOM['actions'] == 1]['dates'], y=df_robot_actions_sim_id_RANDOM[df_robot_actions_sim_id_RANDOM['actions'] == 1]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#228b22', legendgroup = '4'), row=4, col=1)
    # fig.add_trace(go.Scatter(name='Short', x=df_robot_actions_sim_id_RANDOM[df_robot_actions_sim_id_RANDOM['actions'] == 0]['dates'], y=df_robot_actions_sim_id_RANDOM[df_robot_actions_sim_id_RANDOM['actions'] == 0]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#c83232', legendgroup = '4'), row=4, col=1)

    fig.update_xaxes(title_text = "Dates", title_standoff = 25)

    fig.update_yaxes(title_text = "", title_standoff = 25)

    fig.update_layout(title=str_title, title_x=0.5, width=1200, height=800, legend_tracegroupgap=125)
    fig.show()

In [25]:
plot_robot_actions_all_models(config_file_name_without_extension, "", median_sim_id_PPO, "", "")

In [26]:
def get_and_process_trader_robot_actions_and_env_data_all_sim_id(config_file_name_without_extension):
    try:
        
        # Get the data
        db_address = 'sqlite:///../results/data/db/trader.db'
        engine = create_engine(db_address, echo=False)
        sqlite_connection = engine.connect()

        input_tbl_name = "tbl_trader_bernardo_robot_actions_and_env" + "_" + config_file_name_without_extension
        
        sql_command = "SELECT * FROM " + input_tbl_name + " AS t1 "
        
        sql_append = "LEFT JOIN (SELECT sim_id, model_type FROM tbl_trader_bernardo_simulation_results_" + config_file_name_without_extension + ") AS t2 ON t1.sim_id = t2.sim_id"
        
        sql_command = sql_command + sql_append

        df = pd.read_sql(sql_command, sqlite_connection)

        sqlite_connection.close()

        return df

    except Exception as e:
        print(e)

In [27]:
def plot_trader_robot_actions_all_sim_id_all_models(config_file_name_without_extension, sim_id_A2C, sim_id_PPO, sim_id_DQN, sim_id_RANDOM):
    
    df_robot_actions_and_env_all_sim_id = get_and_process_trader_robot_actions_and_env_data_all_sim_id(config_file_name_without_extension)

    # df_robot_actions_and_env_all_sim_id_A2C = df_robot_actions_and_env_all_sim_id[df_robot_actions_and_env_all_sim_id['model_type'] == 'A2C']
    df_robot_actions_and_env_all_sim_id_PPO = df_robot_actions_and_env_all_sim_id[df_robot_actions_and_env_all_sim_id['model_type'] == 'PPO']
    # df_robot_actions_and_env_all_sim_id_DQN = df_robot_actions_and_env_all_sim_id[df_robot_actions_and_env_all_sim_id['model_type'] == 'DQN']
    # df_robot_actions_and_env_all_sim_id_RANDOM = df_robot_actions_and_env_all_sim_id[df_robot_actions_and_env_all_sim_id['model_type'] == 'RANDOM']    

    # df_model_type_action_count_A2C = df_robot_actions_and_env_all_sim_id_A2C.assign(val=1).pivot_table(values='val', index=['dates'], columns=['model_type', 'actions'], aggfunc='count').stack(0).reset_index()
    # df_model_type_action_count_A2C.rename(columns = {0.0:'action_short_count', 1.0:'action_long_count'}, inplace = True)

    df_model_type_action_count_PPO = df_robot_actions_and_env_all_sim_id_PPO.assign(val=1).pivot_table(values='val', index=['dates'], columns=['model_type', 'actions'], aggfunc='count').stack(0).reset_index()
    df_model_type_action_count_PPO.rename(columns = {0.0:'action_short_count', 1.0:'action_long_count'}, inplace = True)

    # df_model_type_action_count_DQN = df_robot_actions_and_env_all_sim_id_DQN.assign(val=1).pivot_table(values='val', index=['dates'], columns=['model_type', 'actions'], aggfunc='count').stack(0).reset_index()
    # df_model_type_action_count_DQN.rename(columns = {0.0:'action_short_count', 1.0:'action_long_count'}, inplace = True)

    # df_model_type_action_count_RANDOM = df_robot_actions_and_env_all_sim_id_RANDOM.assign(val=1).pivot_table(values='val', index=['dates'], columns=['model_type', 'actions'], aggfunc='count').stack(0).reset_index()
    # df_model_type_action_count_RANDOM.rename(columns = {0.0:'action_short_count', 1.0:'action_long_count'}, inplace = True)        

    # df_robot_actions_sim_id_A2C = get_and_process_trader_robot_actions_and_env_data_one_sim_id(config_file_name_without_extension, sim_id_A2C)
    df_robot_actions_sim_id_PPO = get_and_process_trader_robot_actions_and_env_data_one_sim_id(config_file_name_without_extension, sim_id_PPO)
    # df_robot_actions_sim_id_DQN = get_and_process_trader_robot_actions_and_env_data_one_sim_id(config_file_name_without_extension, sim_id_DQN)
    # df_robot_actions_sim_id_RANDOM = get_and_process_trader_robot_actions_and_env_data_one_sim_id(config_file_name_without_extension, sim_id_RANDOM)

    # df_simulation_sim_id_A2C = df_trader_results[df_trader_results['sim_id'] == sim_id_A2C]
    df_simulation_sim_id_PPO = df_trader_results[df_trader_results['sim_id'] == sim_id_PPO]
    # df_simulation_sim_id_DQN = df_trader_results[df_trader_results['sim_id'] == sim_id_DQN]
    # df_simulation_sim_id_RANDOM = df_trader_results[df_trader_results['sim_id'] == sim_id_RANDOM]

    # str_title_A2C = f"Median sim_id: {sim_id_A2C} | model_type: {df_simulation_sim_id_A2C.model_type.values[0]} | Total Profit percentage {round(df_simulation_sim_id_A2C.fmt_total_profit_percentage.values[0],2)}%"
    str_title_PPO = f"Median sim_id: {sim_id_PPO} | model_type: {df_simulation_sim_id_PPO.model_type.values[0]} | Total Profit percentage {round(df_simulation_sim_id_PPO.fmt_total_profit_percentage.values[0],2)}%"
    # str_title_DQN = f"Median sim_id: {sim_id_DQN} | model_type: {df_simulation_sim_id_DQN.model_type.values[0]} | Total Profit percentage {round(df_simulation_sim_id_DQN.fmt_total_profit_percentage.values[0],2)}%"
    # str_title_RANDOM = f"Median sim_id: {sim_id_RANDOM} | model_type: {df_simulation_sim_id_RANDOM.model_type.values[0]} | Total Profit percentage {round(df_simulation_sim_id_RANDOM.fmt_total_profit_percentage.values[0],2)}%"

    str_title = f"Median Total Profit Results x Overall view of actions (across all the simulations)"

    fig = make_subplots(rows=4, cols=1, subplot_titles=("", str_title_PPO, "", ""), shared_xaxes='all', shared_yaxes='all', specs=[[{"secondary_y": True}], [{"secondary_y": True}], [{"secondary_y": True}], [{"secondary_y": True}]]) 

    # fig.add_trace(go.Scatter(name='Price', x=df_robot_actions_sim_id_A2C["dates"], y=df_robot_actions_sim_id_A2C["prices"], mode='lines', marker_color='#969696', legendgroup = '1'), row=1, col=1, secondary_y=True)
    # fig.add_trace(go.Scatter(name='Long', x=df_robot_actions_sim_id_A2C[df_robot_actions_sim_id_A2C['actions'] == 1]['dates'], y=df_robot_actions_sim_id_A2C[df_robot_actions_sim_id_A2C['actions'] == 1]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#228b22', legendgroup = '1'), row=1, col=1, secondary_y=True)
    # fig.add_trace(go.Scatter(name='Short', x=df_robot_actions_sim_id_A2C[df_robot_actions_sim_id_A2C['actions'] == 0]['dates'], y=df_robot_actions_sim_id_A2C[df_robot_actions_sim_id_A2C['actions'] == 0]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#c83232', legendgroup = '1'), row=1, col=1, secondary_y=True)

    # fig.add_trace(go.Bar(x=df_model_type_action_count_A2C['dates'], y=df_model_type_action_count_A2C['action_long_count'], name='Long', marker_color=' #b3ffb3', legendgroup = '1'), row=1, col=1, secondary_y=False)
    # fig.add_trace(go.Bar(x=df_model_type_action_count_A2C['dates'], y=df_model_type_action_count_A2C['action_short_count'], name='Short', marker_color='#ffb2b2', legendgroup = '1'), row=1, col=1, secondary_y=False)

    fig.add_trace(go.Scatter(name='Price', x=df_robot_actions_sim_id_PPO["dates"], y=df_robot_actions_sim_id_PPO["prices"], mode='lines', marker_color='#969696', legendgroup = '2'), row=2, col=1, secondary_y=True)
    fig.add_trace(go.Scatter(name='Long', x=df_robot_actions_sim_id_PPO[df_robot_actions_sim_id_PPO['actions'] == 1]['dates'], y=df_robot_actions_sim_id_PPO[df_robot_actions_sim_id_PPO['actions'] == 1]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#228b22', legendgroup = '2'), row=2, col=1, secondary_y=True)
    fig.add_trace(go.Scatter(name='Short', x=df_robot_actions_sim_id_PPO[df_robot_actions_sim_id_PPO['actions'] == 0]['dates'], y=df_robot_actions_sim_id_PPO[df_robot_actions_sim_id_PPO['actions'] == 0]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#c83232', legendgroup = '2'), row=2, col=1, secondary_y=True)

    fig.add_trace(go.Bar(x=df_model_type_action_count_PPO['dates'], y=df_model_type_action_count_PPO['action_long_count'], name='Long', marker_color=' #b3ffb3', legendgroup = '2'), row=2, col=1, secondary_y=False)
    fig.add_trace(go.Bar(x=df_model_type_action_count_PPO['dates'], y=df_model_type_action_count_PPO['action_short_count'], name='Short', marker_color='#ffb2b2', legendgroup = '2'), row=2, col=1, secondary_y=False)

    # fig.add_trace(go.Scatter(name='Price', x=df_robot_actions_sim_id_DQN["dates"], y=df_robot_actions_sim_id_DQN["prices"], mode='lines', marker_color='#969696', legendgroup = '3'), row=3, col=1, secondary_y=True)
    # fig.add_trace(go.Scatter(name='Long', x=df_robot_actions_sim_id_DQN[df_robot_actions_sim_id_DQN['actions'] == 1]['dates'], y=df_robot_actions_sim_id_DQN[df_robot_actions_sim_id_DQN['actions'] == 1]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#228b22', legendgroup = '3'), row=3, col=1, secondary_y=True)
    # fig.add_trace(go.Scatter(name='Short', x=df_robot_actions_sim_id_DQN[df_robot_actions_sim_id_DQN['actions'] == 0]['dates'], y=df_robot_actions_sim_id_DQN[df_robot_actions_sim_id_DQN['actions'] == 0]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#c83232', legendgroup = '3'), row=3, col=1, secondary_y=True)

    # fig.add_trace(go.Bar(x=df_model_type_action_count_DQN['dates'], y=df_model_type_action_count_DQN['action_long_count'], name='Long', marker_color=' #b3ffb3', legendgroup = '3'), row=3, col=1, secondary_y=False)
    # fig.add_trace(go.Bar(x=df_model_type_action_count_DQN['dates'], y=df_model_type_action_count_DQN['action_short_count'], name='Short', marker_color='#ffb2b2', legendgroup = '3'), row=3, col=1, secondary_y=False)

    # fig.add_trace(go.Scatter(name='Price', x=df_robot_actions_sim_id_RANDOM["dates"], y=df_robot_actions_sim_id_RANDOM["prices"], mode='lines', marker_color='#969696', legendgroup = '4'), row=4, col=1, secondary_y=True)
    # fig.add_trace(go.Scatter(name='Long', x=df_robot_actions_sim_id_RANDOM[df_robot_actions_sim_id_RANDOM['actions'] == 1]['dates'], y=df_robot_actions_sim_id_RANDOM[df_robot_actions_sim_id_RANDOM['actions'] == 1]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#228b22', legendgroup = '4'), row=4, col=1, secondary_y=True)
    # fig.add_trace(go.Scatter(name='Short', x=df_robot_actions_sim_id_RANDOM[df_robot_actions_sim_id_RANDOM['actions'] == 0]['dates'], y=df_robot_actions_sim_id_RANDOM[df_robot_actions_sim_id_RANDOM['actions'] == 0]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#c83232', legendgroup = '4'), row=4, col=1, secondary_y=True)

    # fig.add_trace(go.Bar(x=df_model_type_action_count_RANDOM['dates'], y=df_model_type_action_count_RANDOM['action_long_count'], name='Long', marker_color=' #b3ffb3', legendgroup = '4'), row=4, col=1, secondary_y=False)
    # fig.add_trace(go.Bar(x=df_model_type_action_count_RANDOM['dates'], y=df_model_type_action_count_RANDOM['action_short_count'], name='Short', marker_color='#ffb2b2', legendgroup = '4'), row=4, col=1, secondary_y=False)

    fig.update_yaxes(title_text="Proportion of trades<br>across all the simulations", secondary_y=False)
    fig.update_yaxes(title_text=f"CCY level with the<br>sim_id actions related", secondary_y=True)

    fig.update_layout(barmode='stack', xaxis={'categoryorder':'category ascending'}, title=str_title, title_x=0.5, width=1200, height=800, legend_tracegroupgap=82, font={'size': 8})

    fig.show()    

In [28]:
plot_trader_robot_actions_all_sim_id_all_models(config_file_name_without_extension, "", median_sim_id_PPO, "", "")

In [81]:
def calc_reward(df):
    
    total_profit_percentage = 1.0
    previous_action = df['actions'].iloc[0]
    previous_price = df['prices'].iloc[0]
    
    for index, row in df.iterrows():
    
        current_action = row['actions']
        current_price = row['prices']
    
        if(previous_action != current_action):

            if(current_action == 0):
                reward = current_price / previous_price
            else:
                reward = previous_price / current_price
            
            total_profit_percentage = total_profit_percentage * reward
            previous_action = row['actions'] 
            previous_price = row['prices']

        df.loc[index,'total_profit_percentage'] = total_profit_percentage
        df.loc[index,'fmt_total_profit_percentage'] = (total_profit_percentage - 1 ) * 100
        
    return df

In [84]:
def plot_trader_robot_actions_all_sim_id_combined_all_models(config_file_name_without_extension):
    
    df_robot_actions_and_env_all_sim_id = get_and_process_trader_robot_actions_and_env_data_all_sim_id(config_file_name_without_extension)
    df_prices = df_robot_actions_and_env_all_sim_id[['dates', 'prices']].drop_duplicates()

    # df_robot_actions_and_env_all_sim_id_A2C = df_robot_actions_and_env_all_sim_id[df_robot_actions_and_env_all_sim_id['model_type'] == 'A2C']
    df_robot_actions_and_env_all_sim_id_PPO = df_robot_actions_and_env_all_sim_id[df_robot_actions_and_env_all_sim_id['model_type'] == 'PPO']
    # df_robot_actions_and_env_all_sim_id_DQN = df_robot_actions_and_env_all_sim_id[df_robot_actions_and_env_all_sim_id['model_type'] == 'DQN']
    # df_robot_actions_and_env_all_sim_id_RANDOM = df_robot_actions_and_env_all_sim_id[df_robot_actions_and_env_all_sim_id['model_type'] == 'RANDOM']    

    # df_model_type_action_count_A2C = df_robot_actions_and_env_all_sim_id_A2C.assign(val=1).pivot_table(values='val', index=['dates'], columns=['model_type', 'actions'], aggfunc='count').stack(0).reset_index()
    # df_model_type_action_count_A2C.rename(columns = {0.0:'action_short_count', 1.0:'action_long_count'}, inplace = True)
    # df_model_type_action_count_A2C['actions'] = (df_model_type_action_count_A2C["action_long_count"]>=df_model_type_action_count_A2C["action_short_count"])*1
    # df_model_type_action_count_A2C = df_model_type_action_count_A2C.merge(df_prices, on='dates', how='left')
    # df_model_type_action_count_A2C = calc_reward(df_model_type_action_count_A2C)

    df_model_type_action_count_PPO = df_robot_actions_and_env_all_sim_id_PPO.assign(val=1).pivot_table(values='val', index=['dates'], columns=['model_type', 'actions'], aggfunc='count').stack(0).reset_index()
    df_model_type_action_count_PPO.rename(columns = {0.0:'action_short_count', 1.0:'action_long_count'}, inplace = True)
    df_model_type_action_count_PPO['actions'] = (df_model_type_action_count_PPO["action_long_count"]>=df_model_type_action_count_PPO["action_short_count"])*1
    df_model_type_action_count_PPO = df_model_type_action_count_PPO.merge(df_prices, on='dates', how='left')
    df_model_type_action_count_PPO = calc_reward(df_model_type_action_count_PPO)

    # df_model_type_action_count_DQN = df_robot_actions_and_env_all_sim_id_DQN.assign(val=1).pivot_table(values='val', index=['dates'], columns=['model_type', 'actions'], aggfunc='count').stack(0).reset_index()
    # df_model_type_action_count_DQN.rename(columns = {0.0:'action_short_count', 1.0:'action_long_count'}, inplace = True)
    # df_model_type_action_count_DQN['actions'] = (df_model_type_action_count_DQN["action_long_count"]>=df_model_type_action_count_DQN["action_short_count"])*1
    # df_model_type_action_count_DQN = df_model_type_action_count_DQN.merge(df_prices, on='dates', how='left')
    # df_model_type_action_count_DQN = calc_reward(df_model_type_action_count_DQN)

    # df_model_type_action_count_RANDOM = df_robot_actions_and_env_all_sim_id_RANDOM.assign(val=1).pivot_table(values='val', index=['dates'], columns=['model_type', 'actions'], aggfunc='count').stack(0).reset_index()
    # df_model_type_action_count_RANDOM.rename(columns = {0.0:'action_short_count', 1.0:'action_long_count'}, inplace = True)
    # df_model_type_action_count_RANDOM['actions'] = (df_model_type_action_count_RANDOM["action_long_count"]>=df_model_type_action_count_RANDOM["action_short_count"])*1
    # df_model_type_action_count_RANDOM = df_model_type_action_count_RANDOM.merge(df_prices, on='dates', how='left')
    # df_model_type_action_count_RANDOM = calc_reward(df_model_type_action_count_RANDOM)

    # str_title_A2C = f"Median sim_id: mix | model_type: A2C | Total Profit percentage : {round(df_model_type_action_count_A2C.fmt_total_profit_percentage.values[-1],2)}%"
    str_title_PPO = f"Median sim_id: mix | model_type: PPO | Total Profit percentage : {round(df_model_type_action_count_PPO.fmt_total_profit_percentage.values[-1],2)}%"
    # str_title_DQN = f"Median sim_id: mix | model_type: DQN | Total Profit percentage : {round(df_model_type_action_count_DQN.fmt_total_profit_percentage.values[-1],2)}%"
    # str_title_RANDOM = f"Median sim_id: mix | model_type: RANDOM | Total Profit percentage : {round(df_model_type_action_count_RANDOM.fmt_total_profit_percentage.values[-1],2)}%"

    str_title = f"Median Total Profit Results x Overall view of actions (across all the simulations)"

    fig = make_subplots(rows=4, cols=1, subplot_titles=("", str_title_PPO, "", ""), shared_xaxes='all', shared_yaxes='all', specs=[[{"secondary_y": True}], [{"secondary_y": True}], [{"secondary_y": True}], [{"secondary_y": True}]]) 

    # fig.add_trace(go.Scatter(name='Price', x=df_model_type_action_count_A2C["dates"], y=df_model_type_action_count_A2C["prices"], mode='lines', marker_color='#969696', legendgroup = '1'), row=1, col=1, secondary_y=True)
    # fig.add_trace(go.Scatter(name='Long', x=df_model_type_action_count_A2C[df_model_type_action_count_A2C['actions'] == 1]['dates'], y=df_model_type_action_count_A2C[df_model_type_action_count_A2C['actions'] == 1]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#228b22', legendgroup = '1'), row=1, col=1, secondary_y=True)
    # fig.add_trace(go.Scatter(name='Short', x=df_model_type_action_count_A2C[df_model_type_action_count_A2C['actions'] == 0]['dates'], y=df_model_type_action_count_A2C[df_model_type_action_count_A2C['actions'] == 0]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#c83232', legendgroup = '1'), row=1, col=1, secondary_y=True)

    # fig.add_trace(go.Bar(x=df_model_type_action_count_A2C['dates'], y=df_model_type_action_count_A2C['action_long_count'], name='Long', marker_color=' #b3ffb3', legendgroup = '1'), row=1, col=1, secondary_y=False)
    # fig.add_trace(go.Bar(x=df_model_type_action_count_A2C['dates'], y=df_model_type_action_count_A2C['action_short_count'], name='Short', marker_color='#ffb2b2', legendgroup = '1'), row=1, col=1, secondary_y=False)

    fig.add_trace(go.Scatter(name='Price', x=df_model_type_action_count_PPO["dates"], y=df_model_type_action_count_PPO["prices"], mode='lines', marker_color='#969696', legendgroup = '2'), row=2, col=1, secondary_y=True)
    fig.add_trace(go.Scatter(name='Long', x=df_model_type_action_count_PPO[df_model_type_action_count_PPO['actions'] == 1]['dates'], y=df_model_type_action_count_PPO[df_model_type_action_count_PPO['actions'] == 1]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#228b22', legendgroup = '2'), row=2, col=1, secondary_y=True)
    fig.add_trace(go.Scatter(name='Short', x=df_model_type_action_count_PPO[df_model_type_action_count_PPO['actions'] == 0]['dates'], y=df_model_type_action_count_PPO[df_model_type_action_count_PPO['actions'] == 0]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#c83232', legendgroup = '2'), row=2, col=1, secondary_y=True)

    fig.add_trace(go.Bar(x=df_model_type_action_count_PPO['dates'], y=df_model_type_action_count_PPO['action_long_count'], name='Long', marker_color=' #b3ffb3', legendgroup = '2'), row=2, col=1, secondary_y=False)
    fig.add_trace(go.Bar(x=df_model_type_action_count_PPO['dates'], y=df_model_type_action_count_PPO['action_short_count'], name='Short', marker_color='#ffb2b2', legendgroup = '2'), row=2, col=1, secondary_y=False)

    # fig.add_trace(go.Scatter(name='Price', x=df_model_type_action_count_DQN["dates"], y=df_model_type_action_count_DQN["prices"], mode='lines', marker_color='#969696', legendgroup = '3'), row=3, col=1, secondary_y=True)
    # fig.add_trace(go.Scatter(name='Long', x=df_model_type_action_count_DQN[df_model_type_action_count_DQN['actions'] == 1]['dates'], y=df_model_type_action_count_DQN[df_model_type_action_count_DQN['actions'] == 1]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#228b22', legendgroup = '3'), row=3, col=1, secondary_y=True)
    # fig.add_trace(go.Scatter(name='Short', x=df_model_type_action_count_DQN[df_model_type_action_count_DQN['actions'] == 0]['dates'], y=df_model_type_action_count_DQN[df_model_type_action_count_DQN['actions'] == 0]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#c83232', legendgroup = '3'), row=3, col=1, secondary_y=True)

    # fig.add_trace(go.Bar(x=df_model_type_action_count_DQN['dates'], y=df_model_type_action_count_DQN['action_long_count'], name='Long', marker_color=' #b3ffb3', legendgroup = '3'), row=3, col=1, secondary_y=False)
    # fig.add_trace(go.Bar(x=df_model_type_action_count_DQN['dates'], y=df_model_type_action_count_DQN['action_short_count'], name='Short', marker_color='#ffb2b2', legendgroup = '3'), row=3, col=1, secondary_y=False)

    # fig.add_trace(go.Scatter(name='Price', x=df_model_type_action_count_RANDOM["dates"], y=df_model_type_action_count_RANDOM["prices"], mode='lines', marker_color='#969696', legendgroup = '4'), row=4, col=1, secondary_y=True)
    # fig.add_trace(go.Scatter(name='Long', x=df_model_type_action_count_RANDOM[df_model_type_action_count_RANDOM['actions'] == 1]['dates'], y=df_model_type_action_count_RANDOM[df_model_type_action_count_RANDOM['actions'] == 1]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#228b22', legendgroup = '4'), row=4, col=1, secondary_y=True)
    # fig.add_trace(go.Scatter(name='Short', x=df_model_type_action_count_RANDOM[df_model_type_action_count_RANDOM['actions'] == 0]['dates'], y=df_model_type_action_count_RANDOM[df_model_type_action_count_RANDOM['actions'] == 0]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#c83232', legendgroup = '4'), row=4, col=1, secondary_y=True)

    # fig.add_trace(go.Bar(x=df_model_type_action_count_RANDOM['dates'], y=df_model_type_action_count_RANDOM['action_long_count'], name='Long', marker_color=' #b3ffb3', legendgroup = '4'), row=4, col=1, secondary_y=False)
    # fig.add_trace(go.Bar(x=df_model_type_action_count_RANDOM['dates'], y=df_model_type_action_count_RANDOM['action_short_count'], name='Short', marker_color='#ffb2b2', legendgroup = '4'), row=4, col=1, secondary_y=False)

    fig.update_yaxes(title_text="Proportion of trades<br>across all the simulations", secondary_y=False)
    fig.update_yaxes(title_text=f"CCY level with the<br>sim_id actions related", secondary_y=True)

    fig.update_layout(barmode='stack', xaxis={'categoryorder':'category ascending'}, title=str_title, title_x=0.5, width=1200, height=800, legend_tracegroupgap=82, font={'size': 8})

    fig.show()    

In [85]:
plot_trader_robot_actions_all_sim_id_combined_all_models(config_file_name_without_extension)

In [29]:
# Evaluate the loaded policy
mean_reward, std_reward = evaluate_policy(saved_policy, env, n_eval_episodes=10, deterministic=True)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=225.05 +/- 0.0


In [53]:
df_robot_actions_and_env_all_sim_id = get_and_process_trader_robot_actions_and_env_data_all_sim_id(config_file_name_without_extension)

In [77]:
df_robot_actions_and_env_all_sim_id

,dates,prices,actions,total_reward_cash,total_profit_percentage,fmt_total_profit_percentage,num_of_trades,sim_id,sim_id,model_type
0,2022-12-27 00:00:00.000000,244.3,NaN,0.0,0.000000,0.000000,0.0,1,1,PPO
1,2022-12-28 00:00:00.000000,246.7,NaN,0.0,0.000000,0.000000,0.0,1,1,PPO
2,2022-12-29 00:00:00.000000,244.5,NaN,0.0,0.000000,0.000000,0.0,1,1,PPO
3,2022-12-30 00:00:00.000000,246.3,NaN,0.0,0.000000,0.000000,0.0,1,1,PPO
4,2022-12-31 00:00:00.000000,245.8,NaN,0.0,0.000000,0.000000,0.0,1,1,PPO
...,...,...,...,...,...,...,...,...,...,...
48995,2023-02-09 00:00:00.000000,328.2,0.0,43.8,1.180923,18.092293,7.0,1000,1000,PPO
48996,2023-02-10 00:00:00.000000,306.2,0.0,43.8,1.180923,18.092293,7.0,1000,1000,PPO
48997,2023-02-11 00:00:00.000000,305.8,0.0,43.8,1.180923,18.092293,7.0,1000,1000,PPO
48998,2023-02-12 00:00:00.000000,310.4,0.0,43.8,1.180923,18.092293,7.0,1000,1000,PPO


In [54]:
df_prices = df_robot_actions_and_env_all_sim_id[['dates', 'prices']].drop_duplicates()

In [55]:
df_robot_actions_and_env_all_sim_id_PPO = df_robot_actions_and_env_all_sim_id[df_robot_actions_and_env_all_sim_id['model_type'] == 'PPO']

df_model_type_action_count_PPO = df_robot_actions_and_env_all_sim_id_PPO.assign(val=1).pivot_table(values='val', index=['dates'], columns=['model_type', 'actions'], aggfunc='count').stack(0).reset_index()
df_model_type_action_count_PPO.rename(columns = {0.0:'action_short_count', 1.0:'action_long_count'}, inplace = True)
df_model_type_action_count_PPO['actions'] = (df_model_type_action_count_PPO["action_long_count"]>=df_model_type_action_count_PPO["action_short_count"])*1
df_model_type_action_count_PPO = df_model_type_action_count_PPO.merge(df_prices, on='dates', how='left')

In [79]:
x = calc_reward(df_model_type_action_count_PPO)

In [80]:
x

,dates,model_type,action_short_count,action_long_count,actions,prices,total_rewards,total_profit_percentage,fmt_total_profit_percentage
0,2023-01-02 00:00:00.000000,PPO,46,954,1,244.4,1.000000,1.000000,0.000000
1,2023-01-03 00:00:00.000000,PPO,46,954,1,245.2,1.000000,1.000000,0.000000
2,2023-01-04 00:00:00.000000,PPO,26,974,1,246.1,1.000000,1.000000,0.000000
3,2023-01-05 00:00:00.000000,PPO,23,977,1,258.9,1.000000,1.000000,0.000000
4,2023-01-06 00:00:00.000000,PPO,36,964,1,256.7,1.000000,1.000000,0.000000
5,2023-01-07 00:00:00.000000,PPO,134,866,1,259.9,1.000000,1.000000,0.000000
6,2023-01-08 00:00:00.000000,PPO,83,917,1,261.3,1.000000,1.000000,0.000000
7,2023-01-09 00:00:00.000000,PPO,13,987,1,274.9,1.000000,1.000000,0.000000
8,2023-01-10 00:00:00.000000,PPO,47,953,1,272.6,1.000000,1.000000,0.000000
9,2023-01-11 00:00:00.000000,PPO,22,978,1,277.1,1.000000,1.000000,0.000000


In [43]:
df_model_type_action_count_PPO

,dates,model_type,action_short_count,action_long_count,action,prices
0,2023-01-02 00:00:00.000000,PPO,46,954,1,244.4
1,2023-01-03 00:00:00.000000,PPO,46,954,1,245.2
2,2023-01-04 00:00:00.000000,PPO,26,974,1,246.1
3,2023-01-05 00:00:00.000000,PPO,23,977,1,258.9
4,2023-01-06 00:00:00.000000,PPO,36,964,1,256.7
5,2023-01-07 00:00:00.000000,PPO,134,866,1,259.9
6,2023-01-08 00:00:00.000000,PPO,83,917,1,261.3
7,2023-01-09 00:00:00.000000,PPO,13,987,1,274.9
8,2023-01-10 00:00:00.000000,PPO,47,953,1,272.6
9,2023-01-11 00:00:00.000000,PPO,22,978,1,277.1
